### Print answers

In [7]:
import json 

path_kg = "ElementKG/answers/answers_GraphRAG_KG.json"
path_kg_onto = "ElementKG/answers/answers_GraphRAG_KG_Ontology.json"

ANTHROPIC_API_KEY = "<YOUR_ANTHROPIC_API_KEY_HERE>"
MODEL = "claude-3-5-sonnet-20240620"
TEMPERATURE = 0

criteria = [
    'Comprehensiveness: How much detail does the answer provide to cover all aspects and details of the question?',
    'Diversity: How varied and rich is the answer in providing different perspectives and insights on the question?',
    'Empowerment: How well does the answer help the reader understand and make informed judgements about the topic?',
    'Directness: How specifically and clearly does the answer address the question?'
]
CRITERIA = 0 # 0: Comprehensiveness, 1: Diversity, 2: Empowerment, 3: Directness

In [4]:
data_kg = json.load(open(path_kg))
data_onto = json.load(open(path_kg_onto))

for i in range(len(data_kg)):

    print('='*50, " QUESTION ", i, '='*50)
    print()
    res = data_kg[i][0].split('\n')
    res = ["\t" + x for x in res if x]
    res = '\n'.join(res)
    print(res)
    print()
    print('='*50, " ANSWER 1 (GRAPHRAG KG)", '='*50)
    print()
    res = data_kg[i][1].split('\n')
    res = ["\t" + x for x in res if x]
    res = '\n'.join(res)
    print(res)
    print()
    print('='*50, " ANSWER 2 (GRAPHRAG KG + ONTOLOGY)", '='*50)
    print()
    res = data_onto[i][1].split('\n')
    res = ["\t" + x for x in res if x]
    res = '\n'.join(res)
    print(res)
    print()
    print('*' * 140)
    print('*' * 140)
    print()

==================================================  QUESTION  0 ==================================================

	What is the general reason why a catalyst is chosen as "active support" in a research paper? 

==================================================  ANSWER 1 (GRAPHRAG KG) ==================================================

	**General Reason for Choosing a Catalyst as Active Support**
	A catalyst is chosen as an "active support" in a research paper because it has been found to enhance the catalytic activity of the material it supports. This is the primary reason, as reported by Analyst 0 with an importance score of 80.
	**Enhancing Catalytic Activity**
	-------------------------------
	The catalyst provides a unique environment that facilitates chemical reactions or participates directly in the reaction mechanism. This is the key factor that makes a catalyst an effective active support.
	**Additional Factors**
	----------------------
	In addition to enhancing catalytic act

In [6]:
import anthropic, os

client = anthropic.Anthropic(
    api_key=ANTHROPIC_API_KEY,
)

def query_anthropic(client, question):
    message = client.messages.create(
        model=MODEL,
        max_tokens=1000,
        temperature=TEMPERATURE,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": question
                    }
                ]
            }
        ]
    )
    return message.content[0].text  

In [ ]:
prompt = """
Given the question and answers below, and given the following criteria: 
{CRITERIA}, 
which of the two answers is better according to the criteria, "ANSWER1" or "ANSWER2"? 
Justify your answer and then answer with " Winner=ANSWER1" or "Winner=ANSWER2"

QUESTION: {QUESTION}

========================================= ANSWER 1 =========================================

{ANSWER1}

========================================= ANSWER 2 =========================================

{ANSWER2}
"""

answers = []
for i in range(len(data_kg)):

    print("Processing question ", i)

    question = data_kg[i][0]
    answer1 = data_kg[i][1]
    answer2 = data_onto[i][1]

    formated = prompt.format(
        CRITERIA=criteria[CRITERIA], 
        QUESTION=question, 
        ANSWER1=answer1, 
        ANSWER2=answer2
    )
    ans = query_anthropic(client, formated)
    if "Winner=ANSWER2".lower() in ans.lower() and "Winner=ANSWER1".lower() not in ans.lower():
        answers.append(2)
    elif "Winner=ANSWER1".lower() in ans.lower() and "Winner=ANSWER2".lower() not in ans.lower():
        answers.append(1)
    else:
        answers.append(0)
        
winner_kg = [a == 1 for a in answers].count(True)
winner_onto = [a == 2 for a in answers].count(True)
no_winner = [a == 0 for a in answers].count(True)

print("Number of times GraphRAG KG won: ", winner_kg)
print("Number of times GraphRAG KG + Ontology won: ", winner_onto)
print("Number of times there was no clear winner: ", no_winner)